# Análisis de Dinámicas de Tráfico (Semana Aleatoria)

Este notebook analiza los patrones de tráfico de diferentes servicios (SMS, Llamadas, Internet). 
**Enfoque:** Se selecciona **una semana aleatoria** del conjunto de datos (de Lunes a Domingo) para analizar los patrones temporales.

In [ ]:
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
from IPython.display import HTML
import numpy as np
import random
import os

# Configuración de visualización
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 8)
plt.rcParams['animation.embed_limit'] = 100  # Aumentar límite a 100MB para permitir animaciones largas

## Carga y Procesamiento Temporal

Cargamos los datos utilizando Dask para eficiencia.

In [ ]:
files = ['../data1.csv/data1.csv', '../data2.csv/data2.csv']
services = ['smsin', 'smsout', 'callin', 'callout', 'internet']

valid_files = [f for f in files if os.path.exists(f)]
if not valid_files:
    raise FileNotFoundError("No se encontraron los archivos de datos.")

print(f"Cargando archivos: {valid_files}")

# Leer CSVs con Dask
ddf = dd.read_csv(valid_files, assume_missing=True)

# --- AGREGACIÓN TEMPORAL ---
ddf_time = ddf[['TimeInterval'] + services]
agg_task = ddf_time.groupby('TimeInterval')[services].sum()

print("Procesando datos temporales...")
with ProgressBar():
    final_df = agg_task.compute()

# Post-procesamiento
final_df = final_df.reset_index()
final_df['Timestamp'] = pd.to_datetime(final_df['TimeInterval'], unit='ms')
final_df = final_df.sort_values('Timestamp')
final_df.set_index('Timestamp', inplace=True)

print("Rango de fechas total disponible:")
print(f"Inicio: {final_df.index.min()}")
print(f"Fin:    {final_df.index.max()}")

## Selección de Semana Aleatoria (Lunes - Domingo)

Seleccionamos una semana al azar dentro del rango de fechas disponible, asegurando que comience en **Lunes**.

In [ ]:
# Rango total de fechas
min_date = final_df.index.min()
max_date = final_df.index.max()

# Encontrar el primer lunes disponible
days_to_first_monday = (7 - min_date.weekday()) % 7
first_monday = min_date + pd.Timedelta(days=days_to_first_monday)
first_monday = first_monday.normalize()

# Calcular cuántas semanas completas hay
total_days = (max_date - first_monday).days
total_weeks = total_days // 7

if total_weeks < 1:
    print("Advertencia: Menos de una semana completa de datos desde el primer lunes. Usando la primera semana disponible ajustada.")
    start_date = first_monday
else:
    # Elegir una semana aleatoria
    random_week_offset = random.randint(0, total_weeks - 1)
    start_date = first_monday + pd.Timedelta(weeks=random_week_offset)

end_date = start_date + pd.Timedelta(days=7)

# Filtrar el DataFrame
week_df = final_df[(final_df.index >= start_date) & (final_df.index < end_date)]

print(f"\nAnalizando semana aleatoria: del {start_date.date()} (Lunes) al {end_date.date()} (Domingo)")
display(week_df.head())

## Visualización Semanal

Gráficas detalladas del comportamiento del tráfico durante la semana seleccionada.

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12), sharex=True)

# Gráfico para Internet
ax1.plot(week_df.index, week_df['internet'], color='purple', label='Internet', linewidth=2)
ax1.set_title(f'Tráfico de Internet (Semana del {start_date.date()})', fontsize=14)
ax1.set_ylabel('Volumen de Datos')
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)

# Gráfico para SMS y Llamadas
colors = {'smsin': 'green', 'smsout': 'lightgreen', 'callin': 'blue', 'callout': 'cyan'}
for service in ['smsin', 'smsout', 'callin', 'callout']:
    ax2.plot(week_df.index, week_df[service], label=service, color=colors.get(service), alpha=0.8)

ax2.set_title(f'Tráfico de SMS y Llamadas (Semana del {start_date.date()})', fontsize=14)
ax2.set_xlabel('Tiempo')
ax2.set_ylabel('Volumen de Eventos')
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.3)

# Forzar límites del eje X
ax2.set_xlim(start_date, end_date)

# Formatear eje X
import matplotlib.dates as mdates
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%A %H:%M'))
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig('../imagenes/internet_sms_call_traffic.png')
plt.show()

## Distribución Espacial 3D (Estática)

Visualización en 3D de la distribución del tráfico acumulado en la cuadrícula urbana (100x100). La altura (eje Z) representa el volumen de tráfico.

In [ ]:
# --- AGREGACIÓN ESPACIAL ---
print("Procesando distribución espacial...")
ddf_spatial = ddf[['GridID'] + services]
spatial_task = ddf_spatial.groupby('GridID')[services].sum()

with ProgressBar():
    spatial_df = spatial_task.compute()

spatial_df['sms_total'] = spatial_df['smsin'] + spatial_df['smsout']
spatial_df['call_total'] = spatial_df['callin'] + spatial_df['callout']

# Función auxiliar para plotear 3D
def plot_3d_surface(data_series, title, filename, cmap='inferno'):
    # --- CREACIÓN DE LA MATRIZ ---
    grid_matrix = np.zeros((100, 100))
    
    # Mapeo GridID -> Matriz
    # Asumiendo que el índice de data_series es GridID
    for grid_id, val in data_series.items():
        if 1 <= grid_id <= 10000:
            r = int((grid_id - 1) // 100)
            c = int((grid_id - 1) % 100)
            grid_matrix[r, c] = val

    # --- PLOT 3D ---
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')

    x = np.arange(0, 100, 1)
    y = np.arange(0, 100, 1)
    X, Y = np.meshgrid(x, y)

    surf = ax.plot_surface(X, Y, grid_matrix, cmap=cmap, edgecolor='none', alpha=0.9)

    ax.set_title(title, fontsize=16)
    ax.set_xlabel('Grid X')
    ax.set_ylabel('Grid Y')
    ax.set_zlabel('Volume')

    fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, label='Volumen')
    ax.view_init(elev=30, azim=45)
    plt.tight_layout()
    plt.savefig(filename)
    plt.show()

# 1. SMS Spatial Distribution
plot_3d_surface(spatial_df['sms_total'], 'Distribución Espacial 3D - SMS', '../imagenes/spatial_sms.png', cmap='inferno')

# 2. Internet Spatial Distribution
plot_3d_surface(spatial_df['internet'], 'Distribución Espacial 3D - Internet', '../imagenes/spatial_internet.png', cmap='viridis')

# 3. Calls Spatial Distribution
plot_3d_surface(spatial_df['call_total'], 'Distribución Espacial 3D - Llamadas', '../imagenes/spatial_calls.png', cmap='plasma')

## Análisis Relacional: Picos, Días y Zonas

NUEVO: Identificamos los días de mayor tráfico, establecemos un umbral y analizamos qué zonas contribuyeron más en esos picos históricos.

In [ ]:
# 1. ANÁLISIS DIARIO GLOBAL
daily_agg = final_df.resample('D').sum()
daily_agg['total_traffic'] = daily_agg[services].sum(axis=1)

# Cálculo de Umbral (Mean + 2*Std)
mean_val = daily_agg['total_traffic'].mean()
std_val = daily_agg['total_traffic'].std()
threshold = mean_val + 2 * std_val

top_5_days = daily_agg.nlargest(5, 'total_traffic')

print(f"Umbral calculado: {threshold:,.2f}")
print("Top 5 Días con más tráfico:")
print(top_5_days['total_traffic'])

# Plot Diario
plt.figure(figsize=(15, 6))
plt.plot(daily_agg.index, daily_agg['total_traffic'], label='Tráfico Total Diario', color='steelblue')
plt.axhline(threshold, color='red', linestyle='--', linewidth=2, label='Umbral (Mean + 2Std)')
plt.scatter(top_5_days.index, top_5_days['total_traffic'], color='red', s=100, zorder=5, label='Picos')
plt.title('Evolución Diaria del Tráfico y Detección de Picos', fontsize=16)
plt.legend()
plt.tight_layout()
plt.savefig('../imagenes/daily_peaks_threshold.png')
plt.show()

In [ ]:
# 2. DRILL-DOWN: Zonas Top en Días Top
print("Desglosando zonas para los 5 días pico...")
top_dates_str = top_5_days.index.strftime('%Y-%m-%d').tolist()

# Filtrar Dask DataFrame original
ddf['DateStr'] = ddf['Timestamp'].dt.strftime('%Y-%m-%d')
ddf_peaks = ddf[ddf['DateStr'].isin(top_dates_str)]
ddf_peaks['total_load'] = ddf_peaks['internet'] + ddf_peaks['smsin'] + ddf_peaks['smsout'] + ddf_peaks['callin'] + ddf_peaks['callout']

# Agrupar
peak_zones_task = ddf_peaks.groupby(['DateStr', 'GridID'])['total_load'].sum()

with ProgressBar():
    peak_zones_df = peak_zones_task.compute().reset_index()

# Preparar datos para Stacked Bar
plot_data = []
for date_str in top_dates_str:
    day_data = peak_zones_df[peak_zones_df['DateStr'] == date_str]
    if day_data.empty: continue
    
    # Top 5 zonas del día
    day_sorted = day_data.sort_values('total_load', ascending=False)
    top_zones = day_sorted.head(5)
    other_load = day_sorted.iloc[5:]['total_load'].sum()
    
    row = {'Date': date_str, 'Others': other_load}
    for _, z in top_zones.iterrows():
        row[f"Zone {int(z['GridID'])}"] = z['total_load']
    plot_data.append(row)

df_plot = pd.DataFrame(plot_data).set_index('Date')

df_plot.plot(kind='bar', stacked=True, figsize=(12, 7), colormap='tab20')
plt.title('Contribución de Zonas en Días de Pico', fontsize=15)
plt.ylabel('Tráfico Total')
plt.xticks(rotation=45)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig('../imagenes/top_days_zones_breakdown.png')
plt.show()

In [ ]:
# 3. ANÁLISIS HORARIO
final_df['Hour'] = final_df.index.hour
final_df['Total'] = final_df[services].sum(axis=1)
hourly_avg = final_df.groupby('Hour')['Total'].mean()

top_hours = hourly_avg.nlargest(5)
print("Mayores horas de tráfico promedio:")
print(top_hours)

plt.figure(figsize=(10, 5))
hourly_avg.plot(kind='bar', color='orange', alpha=0.7)
plt.title('Perfil Horario Promedio (0-24h)', fontsize=14)
plt.ylabel('Tráfico Promedio')
plt.xlabel('Hora')
plt.savefig('../imagenes/hourly_profile.png')
plt.show()

## Evolución Temporal del Tráfico SMS (Animación)

Esta sección genera una animación que muestra cómo "respira" la ciudad. Visualizamos la evolución del tráfico SMS hora a hora durante la semana seleccionada.

In [ ]:
# --- PREPARACIÓN DE DATOS PARA ANIMACIÓN ---
print("Preparando datos para la animación (esto puede tardar)... ")

# Filtrar datos para la semana seleccionada usando Dask
# Convertir TimeInterval a datetime para filtrar
ddf['Timestamp'] = dd.to_datetime(ddf['TimeInterval'], unit='ms')
ddf_week = ddf[(ddf['Timestamp'] >= start_date) & (ddf['Timestamp'] < end_date)]

# Agrupar por Hora y GridID para reducir el tamaño de los datos
# Redondeamos el timestamp a la hora más cercana
ddf_week['Hour'] = ddf_week['Timestamp'].dt.floor('h')

# Sumar SMS (in + out)
ddf_week['sms_total'] = ddf_week['smsin'] + ddf_week['smsout']

# Agrupar
anim_task = ddf_week.groupby(['Hour', 'GridID'])['sms_total'].sum()

with ProgressBar():
    anim_df = anim_task.compute().reset_index()

print("Datos procesados. Generando animación...")

# Obtener lista de horas únicas ordenadas
hours = sorted(anim_df['Hour'].unique())

# Configurar figura 3D
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('Grid X')
ax.set_ylabel('Grid Y')
ax.set_zlabel('SMS Volume')
ax.set_zlim(0, anim_df['sms_total'].max() * 0.8) # Fijar eje Z para estabilidad visual

x = np.arange(0, 100, 1)
y = np.arange(0, 100, 1)
X, Y = np.meshgrid(x, y)

# Función de actualización para la animación
def update_plot(frame_idx):
    ax.clear()
    current_hour = hours[frame_idx]
    
    # Filtrar datos para la hora actual
    hour_data = anim_df[anim_df['Hour'] == current_hour]
    
    # Construir matriz
    grid_matrix = np.zeros((100, 100))
    for _, row in hour_data.iterrows():
        grid_id = int(row['GridID'])
        if 1 <= grid_id <= 10000:
            r = (grid_id - 1) // 100
            c = (grid_id - 1) % 100
            grid_matrix[r, c] = row['sms_total']
            
    # Plotear superficie
    surf = ax.plot_surface(X, Y, grid_matrix, cmap='inferno', edgecolor='none')
    ax.set_title(f'Tráfico SMS - {current_hour.strftime("%A %H:%M")}', fontsize=16)
    ax.set_zlim(0, anim_df['sms_total'].max() * 0.8)
    ax.view_init(elev=30, azim=45)
    return surf,

# Crear animación
ani = animation.FuncAnimation(fig, update_plot, frames=len(hours), interval=200, blit=False)

# Mostrar animación en el notebook
plt.close() # Cerrar plot estático inicial
HTML(ani.to_jshtml())

In [ ]:
import datetime

report_content = f"""# Reporte de Análisis de Tráfico Avanzado\nFecha de generación: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n## 1. Picos y Umbrales\n![Picos Diarios](imagenes/daily_peaks_threshold.png)\n\n## 2. Zonas en Días Pico\n![Zonas Top](imagenes/top_days_zones_breakdown.png)\n\n## 3. Perfil Horario\n![Horas Top](imagenes/hourly_profile.png)\n\n## 4. Análisis Semanal (Aleatorio)\n![Semana](imagenes/internet_sms_call_traffic.png)\n\n## 5. Distribución Espacial General\n![Mapa 3D](imagenes/spatial_internet.png)\n"""

with open('../analysis_report_advanced.md', 'w', encoding='utf-8') as f:
    f.write(report_content)

print("Reporte actualizado generado: ../analysis_report_advanced.md")